In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv("../../data/data_kosice.csv")
df.head()

,title,link,date,price,housing,housing_category,city_area,housing_type,housing_state,living_area,land_area,price_per_meter,ad_id,lat,long
0,2 izb. Byt v osobnom vlastníctve - Aténska ul....,https://www.nehnutelnosti.sk/5055114/2-izb-byt...,31.03.2023,121500,Byty,2 izbový byt,Košice I - Sídlisko Ťahanovce,Predaj,Pôvodný stav,52,0,2336.54,5055114,48.758589,21.263942
1,"4 izb. byt pri Novej Nemocnici, 3.posch., lodž...",https://www.nehnutelnosti.sk/5055771/4-izb-byt...,31.03.2023,159900,Byty,4 izbový byt,Košice II - Západ,Predaj,Pôvodný stav,72,0,2220.83,5055771,48.721930,21.231879
2,"4-izb byt, NOVOSTAVBA Fatranská, Nová Terasa, ...",https://www.nehnutelnosti.sk/5002313/4-izb-byt...,31.03.2023,369000,Byty,4 izbový byt,Košice II - Západ,Predaj,Novostavba,102,0,3617.65,5002313,48.723805,21.240138
3,"nadštandardný 1,5-izb pražský byt, 47m2 + logg...",https://www.nehnutelnosti.sk/5048783/nadstanda...,31.03.2023,132500,Byty,2 izbový byt,Košice II - Západ,Predaj,Kompletná rekonštrukcia,47,0,2819.15,5048783,48.716845,21.239194
4,"Arvin & Benet | Slnečný, stavebný pozemok so v...",https://www.nehnutelnosti.sk/5039704/arvin-ben...,31.03.2023,250000,Pozemky,Pozemok pre rod. domy,Košice IV - Krásna,Predaj,NaN,0,692,361.27,5039704,48.666724,21.318585


## Cleaning rows
we are interested only in ads, that offer flat for sell

In [3]:
df = df[df["housing"] == "Byty"]
df = df[df["housing_type"] == "Predaj"]

df.drop(df.loc[df["price"] == "Cenadohodou"].index, inplace=True)
df.drop(df.loc[df["price"] == "InfovRK"].index, inplace=True)

df["living_area"] = [str(row).replace(',', '.') for row in df["living_area"]]
df["land_area"] = [str(row).replace(',', '.') for row in df["land_area"]]
df["price"] = [str(row).replace(',', '.') for row in df["price"]]
df["city_area"] = [str(row).replace('Košice I - Sídlisko Ťahanovce', 'Košice I - Ťahanovce') for row in df["city_area"]]


df["title"] = df["title"].astype("string")
df["link"] = df["link"].astype("string")
df["date"] = df["date"].astype("string")
df["price"] = df["price"].astype("float64")
df["housing"] = df["housing"].astype("category")
df["housing_category"] = df["housing_category"].astype("category")
df["city_area"] = df["city_area"].astype("category")
df["housing_type"] = df["housing_type"].astype("category")
df["living_area"] = df["living_area"].astype("float64")
df["land_area"] = df["land_area"].astype("float64")

In [4]:
df.info()
# 956 rows of data

<class 'pandas.core.frame.DataFrame'>
Index: 956 entries, 0 to 2015
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   title             956 non-null    string  
 1   link              956 non-null    string  
 2   date              956 non-null    string  
 3   price             956 non-null    float64 
 4   housing           956 non-null    category
 5   housing_category  956 non-null    category
 6   city_area         956 non-null    category
 7   housing_type      956 non-null    category
 8   housing_state     956 non-null    object  
 9   living_area       956 non-null    float64 
 10  land_area         956 non-null    float64 
 11  price_per_meter   956 non-null    float64 
 12  ad_id             956 non-null    int64   
 13  lat               956 non-null    float64 
 14  long              956 non-null    float64 
dtypes: category(4), float64(6), int64(1), object(1), string(3)
memory usage: 94.6+

## Visualizing data

In [16]:
boxplot = px.box(df, y="price", hover_data="ad_id")
boxplot.show()

histogram = px.histogram(df, x="price")
histogram.show()

we can see that, we have got a lot of upper outliers, that may be okay, because price for big falt in the downtown is, of course more expensive, but we should get rid of that one outlier that has price "1". That may be wrong input

In [11]:
df.drop(df.loc[df["ad_id"] == 5063534].index, inplace=True)

In [22]:
correlation_matrix = df.corr(numeric_only=True)
heatmap = px.imshow(correlation_matrix, text_auto=True)
heatmap.show()

- We can see, that only **living_area** atribute has strong correlation to price, our target attribute. We are using neural network, so it should figure out which atributes are important for best results, but we do not need those attrbutes to be inputed by users, therefore user experince should be better.
- We would not use price_per_meter anyway, because that is calculated from our target atribute price and ad_id is just an id

## Which city area is most expensive?

In [61]:
df["city_area"].value_counts()

city_area
Košice II - Západ                   199
Košice I - Staré Mesto              183
Košice III - Dargovských Hrdinov    123
Košice I - Sever                     98
Košice IV - Juh                      95
Košice IV - Nad jazerom              80
Košice II - Sídlisko KVP             69
Košice I - Ťahanovce                 66
Košice II - Šaca                     21
Košice III - Košická Nová Ves         9
Košice                                5
Košice IV - Vyšné Opátske             3
Košice II - Lorinčík                  2
Košice I - Kavečany                   1
Košice IV - Krásna                    1
Name: count, dtype: int64

we should remove city areas with really small sample of flats, let's say < 5.

In [62]:
tmpdf = pd.DataFrame( data = {"city_area" : df["city_area"], "price" : df["price"]})
tmpdf.drop(df.loc[df["city_area"] == "Košice IV - Vyšné Opátske"].index, inplace=True)
tmpdf.drop(df.loc[df["city_area"] == "Košice II - Lorinčík"].index, inplace=True)
tmpdf.drop(df.loc[df["city_area"] == "Košice I - Kavečany"].index, inplace=True)
tmpdf.drop(df.loc[df["city_area"] == "Košice IV - Krásna"].index, inplace=True)

In [63]:
# average
most_expensive = tmpdf.groupby(['city_area']).mean()
barplot = px.bar(most_expensive, title="Average price of flat by city area", text_auto='.3s')
barplot.update_layout(yaxis_title='average price', showlegend=False)
barplot.show()

# median
most_expensive_medians = tmpdf.groupby(['city_area']).median()
barplot_medians = px.bar(most_expensive_medians, title="Median price of flat by city area", text_auto='.3s')
barplot_medians.update_layout(yaxis_title='median price', showlegend=False)
barplot_medians.show()

Košice III - Košická Nová Ves is the most expensive city area, but there are only 9 rows with this city area, should we take this into consideration?

## Key takeways for next steps:
- delete rows with small price < 10, those are probably wrong inputs
- only living_area numeric attribute has strong correlation with price, we do not need other numeric attributes from users
- remove rows with city areas that have less than 5 counts